### 特征转换

#### 1. MinMaxScaler

MinMaxScaler将向量行数字调整到一个特定的范围（例如：[0,1]）

In [12]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("TransformerExample")\
        .getOrCreate()

In [14]:
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])

In [15]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

In [16]:
scalerModel = scaler.fit(dataFrame)

In [17]:
scaledData = scalerModel.transform(dataFrame)

In [18]:
print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))

Features scaled to range: [0.000000, 1.000000]


In [19]:
scaledData.select("features", "scaledFeatures").show()

+--------------+--------------+
|      features|scaledFeatures|
+--------------+--------------+
|[1.0,0.1,-1.0]|     (3,[],[])|
| [2.0,1.1,1.0]| [0.5,0.1,0.5]|
|[3.0,10.1,3.0]| [1.0,1.0,1.0]|
+--------------+--------------+



#### 2. Normalizer

#### 3. OneHotEncoder

#### 4. PCA

#### 5. QuantileDiscretizer